In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
test_data.head(5)

In [ ]:
train_data.head(10)

In [ ]:
columns_to_drop = ['id', 'outcome']
target = train_data['outcome']

train1 = train_data.copy()
train1.drop(columns=columns_to_drop, inplace=True)
test1 = test_data.copy()
test1.drop(columns='id', inplace=True)


print('The shape of the train data:', train1.shape)
print('The shape of the test data:', test1.shape)

In [ ]:
target.head(5)

In [ ]:
def summary(df):
    sum = pd.DataFrame(df.dtypes, columns=['dtypes'])
    sum['missing#'] = df.isna().sum()
    sum['missing%'] = (df.isna().sum())/len(df)
    sum['uniques'] = df.nunique().values
    sum['count'] = df.count().values
    #sum['skew'] = df.skew().values
    return sum

summary(train1).style.background_gradient(cmap='Blues')

### Based on top analysis

* No missing value is observed.
* hospital number is likely the ID of hospital
* dtypes object -> need to convert the object through encoding


In [ ]:
#grouping based on domain knowledge and analysis

num_var = [column for column in train_data.columns if train_data[column].nunique() > 10]

bin_var = [column for column in train_data.columns if train_data[column].nunique() == 2]
# for age -> only adult and young

cat_var = ['temp_of_extremities', 'peripheral_pulse', 'mucous_membrane','capillary_refill_time','pain',
'peristalsis','abdominal_distention','nasogastric_tube','nasogastric_reflux','rectal_exam_feces','abdomen',
'abdomo_appearance','lesion_2','surgery', 'age', 'surgical_lesion', 'lesion_3', 'cp_data']



In [ ]:
# Preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler, PowerTransformer, QuantileTransformer, OrdinalEncoder, LabelEncoder
from sklearn.impute import SimpleImputer

# Data preprocessing, code sourse: https://www.kaggle.com/code/mattop/ps-s3-e22-xgboost-preprocessing
def preprocessing(df, le_cols, ohe_cols):

    # Label Encoding for binary cols
    le = LabelEncoder()
    for col in le_cols:
        df[col] = le.fit_transform(df[col])

    # OneHot Encoding for category cols
    df = pd.get_dummies(df, columns = ohe_cols)

    df["pain"] = df["pain"].replace('slight', 'moderate')
    df["peristalsis"] = df["peristalsis"].replace('distend_small', 'normal')
    df["rectal_exam_feces"] = df["rectal_exam_feces"].replace('serosanguious', 'absent')
    df["nasogastric_reflux"] = df["nasogastric_reflux"].replace('slight', 'none')

    df["temp_of_extremities"] = df["temp_of_extremities"].fillna("normal").map({'cold': 0, 'cool': 1, 'normal': 2, 'warm': 3})
    df["peripheral_pulse"] = df["peripheral_pulse"].fillna("normal").map({'absent': 0, 'reduced': 1, 'normal': 2, 'increased': 3})
    df["capillary_refill_time"] = df["capillary_refill_time"].fillna("3").map({'less_3_sec': 0, '3': 1, 'more_3_sec': 2})
    df["pain"] = df["pain"].fillna("depressed").map({'alert': 0, 'depressed': 1, 'moderate': 2, 'mild_pain': 3, 'severe_pain': 4, 'extreme_pain': 5})
    df["peristalsis"] = df["peristalsis"].fillna("hypomotile").map({'hypermotile': 0, 'normal': 1, 'hypomotile': 2, 'absent': 3})
    df["abdominal_distention"] = df["abdominal_distention"].fillna("none").map({'none': 0, 'slight': 1, 'moderate': 2, 'severe': 3})
    df["nasogastric_tube"] = df["nasogastric_tube"].fillna("none").map({'none': 0, 'slight': 1, 'significant': 2})
    df["nasogastric_reflux"] = df["nasogastric_reflux"].fillna("none").map({'less_1_liter': 0, 'none': 1, 'more_1_liter': 2})
    df["rectal_exam_feces"] = df["rectal_exam_feces"].fillna("absent").map({'absent': 0, 'decreased': 1, 'normal': 2, 'increased': 3})
    df["abdomen"] = df["abdomen"].fillna("distend_small").map({'normal': 0, 'other': 1, 'firm': 2,'distend_small': 3, 'distend_large': 4})
    df["abdomo_appearance"] = df["abdomo_appearance"].fillna("serosanguious").map({'clear': 0, 'cloudy': 1, 'serosanguious': 2})

    # Imputer
    cols_with_nan = df.columns[df.isna().any()].tolist()
    for feature in cols_with_nan:
        df[feature].fillna(df[feature].mode()[0], inplace=True)

    return df

In [ ]:
train2 = preprocessing(train1, le_cols = ["surgery", "age", "surgical_lesion", "cp_data"], ohe_cols = ["mucous_membrane"])

In [ ]:
y_train = target.copy()
y_train = y_train.map({'died':0,'euthanized':1,'lived':2})


In [ ]:
y_train

In [ ]:
train2

In [ ]:
X_train = train2.copy()

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


# Processed Data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Processed Data Input :
serve as input for both FNN model and Ensemble Method

In [ ]:
train_processed = pd.read_csv("/content/drive/MyDrive/[1]Projects 🏋️/Machine Learning /Regular_processed/train_processed.csv")
val_processed = pd.read_csv("/content/drive/MyDrive/[1]Projects 🏋️/Machine Learning /Regular_processed/valid_processed.csv")

In [ ]:
train_processed.head(5)

,Unnamed: 0,rectal_temp,pulse,respiratory_rate,nasogastric_reflux_ph,packed_cell_volume,total_protein,abdomo_protein,lesion_1,lesion_2,...,pain_is_depressed,pain_is_mild_pain,pain_is_extreme_pain,pain_is_alert,pain_is_severe_pain,pain_is_slight,nasogastric_tube_is_slight,nasogastric_tube_is_none,nasogastric_tube_is_significant,outcome
0,294,1.138854,-0.947666,1.820919,-1.746690,-0.057202,-0.419567,-1.064448,-0.314378,-0.075465,...,1,0,0,0,0,0,1,0,0,2
1,24,-0.129621,1.698669,1.091232,-1.230314,0.702421,1.673013,-0.812647,-0.115465,-0.075465,...,1,0,0,0,0,0,0,1,0,1
2,830,-0.129621,-0.535250,3.645137,0.318815,-1.291591,-0.520820,0.005709,5.019292,-0.075465,...,1,0,0,0,0,0,1,0,0,2
3,888,-0.256468,-0.947666,0.118316,0.318815,0.227656,-0.547071,-0.434944,-0.314378,-0.075465,...,0,1,0,0,0,0,1,0,0,0
4,594,0.885159,-0.672722,-0.854601,1.351568,-0.057202,-0.580823,2.334874,-0.705212,-0.075465,...,0,0,0,1,0,0,1,0,0,2


In [ ]:
# Separate the outcome class from the training data and assign it to y_train
y_train = train_processed['outcome']
X_train = train_processed.drop(columns=['outcome'])

# Separate the outcome class from the validation data and assign it to y_val
y_val = val_processed['outcome']
X_val = val_processed.drop(columns=['outcome'])

# Print the shapes of X_train, y_train, X_val, and y_val to verify
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)

X_train shape: (988, 67)
y_train shape: (988,)
X_val shape: (247, 67)
y_val shape: (247,)


**Building the FNN:**

1. **Define Model Structure:**

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import f1_score
from tqdm import tqdm

class FNNModel:
    def __init__(self):
        self.model = None
        self.history = None

    def build_model(self, input_shape):
        self.model = Sequential([
            Dense(64, activation='relu', input_shape=input_shape),
            Dropout(0.2),  # Add dropout with rate 0.5
            Dense(32, activation='relu'),
            Dropout(0.2),  # Add dropout with rate 0.5
            Dense(3, activation='softmax')  # 3 units for 3-class classification with softmax activation
        ])

        self.model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    def train_model(self, X_train, y_train, X_val, y_val):
        num_epochs = 600
        num_batches = len(X_train) // 32  # Assuming batch size of 32

        for epoch in range(num_epochs):
            # Reset the index of the training data
            X_train.reset_index(drop=True, inplace=True)
            y_train.reset_index(drop=True, inplace=True)

            # Shuffle the training data
            indices = np.random.permutation(len(X_train))
            X_train_shuffled = X_train.iloc[indices]
            y_train_shuffled = y_train.iloc[indices]

            # Train the model for one epoch
            self.model.fit(X_train_shuffled, y_train_shuffled, epochs=1, batch_size=32, verbose=0)

            # Print metrics every 100 epochs
            if (epoch + 1) % 100 == 0:
                # Calculate training metrics
                train_loss, train_accuracy = self.model.evaluate(X_train, y_train, verbose=0)

                # Calculate validation metrics
                val_loss, val_accuracy = self.model.evaluate(X_val, y_val, verbose=0)

                # Calculate F1 score for validation set
                y_pred = self.model.predict(X_val)
                y_pred_classes = np.argmax(y_pred, axis=1)
                f1 = f1_score(y_val, y_pred_classes, average='weighted')

                # Print metrics
                tqdm.write(f'Epoch {epoch+1}/{num_epochs} - '
                           f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f} - '
                           f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, F1 Score: {f1:.4f}')


    def save_model(self, filepath):
      self.model.save(filepath)


In [ ]:
fnn_model = FNNModel()
fnn_model.build_model(input_shape=X_train.shape[1:])
fnn_model.train_model(X_train, y_train, X_val, y_val)

8/8 [==============================] - 0s 5ms/step
Epoch 100/600 - Train Loss: 1.0096, Train Accuracy: 0.4757 - Val Loss: 1.0586, Val Accuracy: 0.4899, F1 Score: 0.3483
8/8 [==============================] - 0s 2ms/step
Epoch 200/600 - Train Loss: 0.8616, Train Accuracy: 0.5901 - Val Loss: 1.1652, Val Accuracy: 0.5142, F1 Score: 0.4669
8/8 [==============================] - 0s 3ms/step
Epoch 300/600 - Train Loss: 0.5836, Train Accuracy: 0.7449 - Val Loss: 1.1331, Val Accuracy: 0.6721, F1 Score: 0.6696
8/8 [==============================] - 0s 2ms/step
Epoch 400/600 - Train Loss: 0.4806, Train Accuracy: 0.7945 - Val Loss: 1.2784, Val Accuracy: 0.6883, F1 Score: 0.6907


KeyboardInterrupt: 

In [ ]:
# Save the trained model to the  directory
fnn_model.save_model('/content/drive/MyDrive/[1]Projects 🏋️/Machine Learning/fnn_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Ensemble Model

####**Continuous Features:**

rectal_temp
pulse
respiratory_rate
nasogastric_reflux_ph
packed_cell_volume
total_protein
abdono_protein

####**Categorical Features:**

lesion_1
lesion_2
lesion_3
age_is_adult
surgical_lesion_is_yes
surgery_is_yes
cp_data_is_no
temp_of_extremities_is_cool
temp_of_extremities_is_cold
temp_of_extremities_is_normal
temp_of_extremities_is_warm
peripheral_pulse_is_reduced
peripheral_pulse_is_normal
peripheral_pulse_is_absent
peripheral_pulse_is_increased
nasogastric_reflux_is_less_1_liter
nasogastric_reflux_is_more_1_liter
nasogastric_reflux_is_none
nasogastric_reflux_is_slight
abdominal_distention_is_slight
abdominal_distention_is_moderate
abdominal_distention_is_none
abdominal_distention_is_severe
capillary_refill_time_is_more_3_sec
capillary_refill_time_is_less_3_sec
capillary_refill_time_is_3
peristalsis_is_absent
peristalsis_is_hypomotile
peristalsis_is_normal
peristalsis_is_hypermotile
peristalsis_is_distend_small
abdomen_is_distend_small
abdomen_is_distend_large
abdomen_is_normal
abdomen_is_firm
abdomen_is_other
abdomo_appearance_is_serosanguious
abdomo_appearance_is_cloudy
abdomo_appearance_is_clear
mucous_membrane_is_dark_cyanotic
mucous_membrane_is_pale_cyanotic
mucous_membrane_is_pale_pink
mucous_membrane_is_normal_pink
mucous_membrane_is_bright_pink
mucous_membrane_is_bright_red
rectal_exam_feces_is_decreased
rectal_exam_feces_is_absent
rectal_exam_feces_is_normal
rectal_exam_feces_is_increased
rectal_exam_feces_is_serosanguious
pain_is_depressed
pain_is_mild_pain
pain_is_extreme_pain
pain_is_alert
pain_is_severe_pain
pain_is_slight
nasogastric_tube_is_slight
nasogastric_tube_is_none
nasogastric_tube_is_significant


In [ ]:
train_processed

,Unnamed: 0,rectal_temp,pulse,respiratory_rate,nasogastric_reflux_ph,packed_cell_volume,total_protein,abdomo_protein,lesion_1,lesion_2,...,pain_is_depressed,pain_is_mild_pain,pain_is_extreme_pain,pain_is_alert,pain_is_severe_pain,pain_is_slight,nasogastric_tube_is_slight,nasogastric_tube_is_none,nasogastric_tube_is_significant,outcome
0,294,1.138854,-0.947666,1.820919,-1.746690,-0.057202,-0.419567,-1.064448,-0.314378,-0.075465,...,1,0,0,0,0,0,1,0,0,2
1,24,-0.129621,1.698669,1.091232,-1.230314,0.702421,1.673013,-0.812647,-0.115465,-0.075465,...,1,0,0,0,0,0,0,1,0,1
2,830,-0.129621,-0.535250,3.645137,0.318815,-1.291591,-0.520820,0.005709,5.019292,-0.075465,...,1,0,0,0,0,0,1,0,0,2
3,888,-0.256468,-0.947666,0.118316,0.318815,0.227656,-0.547071,-0.434944,-0.314378,-0.075465,...,0,1,0,0,0,0,1,0,0,0
4,594,0.885159,-0.672722,-0.854601,1.351568,-0.057202,-0.580823,2.334874,-0.705212,-0.075465,...,0,0,0,1,0,0,1,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983,941,0.124074,-0.947666,-0.368142,0.318815,-0.247108,-0.494569,-0.183142,-0.631608,-0.075465,...,0,0,0,0,0,0,1,0,0,2
984,846,-1.017553,0.564525,-0.124913,0.060627,0.987280,-0.558322,0.194560,-0.115465,-0.075465,...,0,0,1,0,0,0,0,0,0,0
985,989,2.661024,0.220846,0.361545,1.351568,-0.531967,-0.498320,0.761114,0.656451,-0.075465,...,1,0,0,0,0,0,1,0,0,2
986,947,1.646244,1.526829,1.273654,1.351568,-0.626920,-0.532071,-0.623795,0.068544,-0.075465,...,1,0,0,0,0,0,0,1,0,2


In [ ]:
# Identify continuous and categorical features
# Dropping Hospital ID and Outcome
train_processed1 = train_processed.drop(columns=['Unnamed: 0', 'outcome'])

con_features = ['rectal_temp', 'pulse', 'respiratory_rate', 'nasogastric_reflux_ph',
                'packed_cell_volume', 'total_protein', 'abdomo_protein']
cat_features = [col for col in train_processed1.columns if col not in con_features]

# Create new DataFrames for continuous and categorical features
con_train_data = train_processed1[con_features]
cat_train_data = train_processed1[cat_features]

In [ ]:
y_train = train_processed['outcome']
y_train.head(5)

0    2
1    1
2    2
3    0
4    2
Name: outcome, dtype: int64

In [ ]:
# Display the first few rows of each DataFrame to verify
print("Continuous Features:")
print(con_train_data.head())

Continuous Features:
   rectal_temp     pulse  respiratory_rate  nasogastric_reflux_ph  \
0     1.138854 -0.947666          1.820919              -1.746690   
1    -0.129621  1.698669          1.091232              -1.230314   
2    -0.129621 -0.535250          3.645137               0.318815   
3    -0.256468 -0.947666          0.118316               0.318815   
4     0.885159 -0.672722         -0.854601               1.351568   

   packed_cell_volume  total_protein  abdomo_protein  
0           -0.057202      -0.419567       -1.064448  
1            0.702421       1.673013       -0.812647  
2           -1.291591      -0.520820        0.005709  
3            0.227656      -0.547071       -0.434944  
4           -0.057202      -0.580823        2.334874  


In [ ]:
print("\nCategorical Features:")
print(cat_train_data.head())


Categorical Features:
   lesion_1  lesion_2  lesion_3  age_is_adult  surgical_lesion_is_yes  \
0 -0.314378 -0.075465 -0.040275             1                       1   
1 -0.115465 -0.075465 -0.040275             1                       0   
2  5.019292 -0.075465 -0.040275             1                       1   
3 -0.314378 -0.075465 -0.040275             1                       0   
4 -0.705212 -0.075465 -0.040275             1                       0   

   surgery_is_yes  cp_data_is_no  temp_of_extremities_is_cool  \
0               1              1                            1   
1               1              1                            1   
2               1              0                            0   
3               0              1                            0   
4               0              0                            0   

   temp_of_extremities_is_cold  temp_of_extremities_is_normal  ...  \
0                            0                              0  ...   
1      

In [ ]:
# Validation Set Extraction
val_processed1 = val_processed.drop(columns=['Unnamed: 0', 'outcome'])

con_val_features  = ['rectal_temp', 'pulse', 'respiratory_rate', 'nasogastric_reflux_ph',
                'packed_cell_volume', 'total_protein', 'abdomo_protein']
cat_val_features = [col for col in val_processed1.columns if col not in con_features]

# Create new DataFrames for continuous and categorical features
con_val_data = val_processed1[con_val_features]
cat_val_data = val_processed1[cat_val_features]

y_val = val_processed['outcome']

In [ ]:
from tqdm import tqdm
from sklearn.metrics import f1_score
from keras.callbacks import Callback
from keras.layers import Input
from keras.layers import Concatenate
from keras.models import Model
from sklearn.utils import resample

class EnsembleModel:
    def __init__(self, con_features, cat_features, base_model_builder, num_models=10):
        self.con_features = con_features
        self.cat_features = cat_features
        self.base_model_builder = base_model_builder
        self.num_models = num_models
        #two separate model for type of features
        self.con_models = []
        self.cat_models = []

    def build_models(self):
        for _ in range(self.num_models):
            con_model = self.base_model_builder(len(self.con_features))
            cat_model = self.base_model_builder(len(self.cat_features))
            self.con_models.append(con_model)
            self.cat_models.append(cat_model)

    def train_models(self, con_data, cat_data, y_train, con_val_data, cat_val_data, y_val):
        for con_model, cat_model in zip(self.con_models, self.cat_models):
            con_model.fit(con_data, y_train, validation_data=(con_val_data, y_val), verbose=1)
            cat_model.fit(cat_data, y_train, validation_data=(cat_val_data, y_val), verbose=1)

    def train_models_boosting(self, con_data, cat_data, y_train, con_val_data, cat_val_data, y_val):
        num_samples = len(y_train)
        sample_weights = np.ones(num_samples) / num_samples  # Initialize sample weights

        for con_model, cat_model in zip(self.con_models, self.cat_models):
            # Train continuous model with weighted samples
            con_model.fit(con_data, y_train, sample_weight=sample_weights, validation_data=(con_val_data, y_val), verbose=1)
            # Make predictions on training set
            con_train_preds = con_model.predict(con_data)
            # Calculate errors
            con_errors = np.abs(np.argmax(con_train_preds, axis=1) - y_train)

            # Train categorical model with weighted samples
            cat_model.fit(cat_data, y_train, sample_weight=sample_weights, validation_data=(cat_val_data, y_val), verbose=1)
            # Make predictions on training set
            cat_train_preds = cat_model.predict(cat_data)
            # Calculate errors
            cat_errors = np.abs(np.argmax(cat_train_preds, axis=1) - y_train)

            # Combine errors from both models
            combined_errors = con_errors + cat_errors
            # Update sample weights based on combined errors
            sample_weights *= np.exp(combined_errors)  # Adjust weights based on errors

            # Normalize sample weights
            sample_weights /= np.sum(sample_weights)

    def predict(self, con_data, cat_data):
        con_predictions = [con_model.predict(con_data) for con_model in self.con_models]
        cat_predictions = [cat_model.predict(cat_data) for cat_model in self.cat_models]
        return con_predictions, cat_predictions

In [ ]:
# Define base model builder function
def create_base_model(input_shape):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(input_shape,)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Initialize and build ensemble model
ensemble_model = EnsembleModel(con_features, cat_features, create_base_model)
ensemble_model.build_models()

# Train ensemble model
ensemble_model.train_models(con_train_data, cat_train_data, y_train, con_val_data, cat_val_data, y_val)

# Make predictions on validation set
con_val_predictions, cat_val_predictions = ensemble_model.predict(con_val_data, cat_val_data)


# Evaluate model performance using F1-score on validation set
con_val_f1 = f1_score(y_val, np.argmax(np.mean(con_val_predictions, axis=0), axis=1), average='macro')
cat_val_f1 = f1_score(y_val, np.argmax(np.mean(cat_val_predictions, axis=0), axis=1), average='macro')

print("F1-score for continuous features model on validation set:", con_val_f1)
print("F1-score for categorical features model on validation set:", cat_val_f1)



8/8 [==============================] - 0s 3ms/step
F1-score for continuous features model on validation set: 0.5760487461917038
F1-score for categorical features model on validation set: 0.43154354765161873


In [ ]:
import tensorflow as tf
con_train_data = tf.convert_to_tensor(con_train_data)
cat_train_data = tf.convert_to_tensor(cat_train_data)
con_val_data = tf.convert_to_tensor(con_val_data)
cat_val_data = tf.convert_to_tensor(cat_val_data)

In [ ]:
cat_train_data

<tf.Tensor: shape=(988, 59), dtype=float64, numpy=
array([[-0.31437781, -0.07546532, -0.04027481, ...,  1.        ,
         0.        ,  0.        ],
       [-0.11546465, -0.07546532, -0.04027481, ...,  0.        ,
         1.        ,  0.        ],
       [ 5.01929192, -0.07546532, -0.04027481, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [ 0.65645091, -0.07546532, -0.04027481, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.06854383, -0.07546532, -0.04027481, ...,  0.        ,
         1.        ,  0.        ],
       [-0.29947313, -0.07546532, -0.04027481, ...,  1.        ,
         0.        ,  0.        ]])>

In [ ]:
cat_val_data

<tf.Tensor: shape=(247, 59), dtype=float64, numpy=
array([[ 5.01929192, -0.07546532, -0.04027481, ...,  1.        ,
         0.        ,  0.        ],
       [-0.31658592, -0.07546532, -0.04027481, ...,  1.        ,
         0.        ,  0.        ],
       [-0.49838629, -0.07546532, -0.04027481, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.62130529, -0.07546532, -0.04027481, ...,  0.        ,
         1.        ,  0.        ],
       [-0.70521182, -0.07546532, -0.04027481, ...,  1.        ,
         0.        ,  0.        ],
       [-0.29873709, -0.07546532, -0.04027481, ...,  1.        ,
         0.        ,  0.        ]])>

In [ ]:
# Instantiate the ensemble model
ensemble_model = EnsembleModel(con_features=con_features, cat_features=cat_features)

# Build the model
ensemble_model.build_model()

# Compile the model
ensemble_model.compile_model()

# Train the model
ensemble_model.train_model(con_train_data, cat_train_data, y_train, con_val_data, cat_val_data, y_val)




Epoch 1:   0%|          | 0/600 [00:00<?, ?it/s]

Epoch 1/600


Epoch 1:   5%|▌         | 31/600 [14:21<4:23:25, 27.78s/it]



Epoch 1:   0%|          | 1/600 [00:03<31:00,  3.11s/it]

20/31 [==================>...........] - ETA: 0s - loss: 1.0487 - accuracy: 0.4203  

Epoch 1:   5%|▌         | 31/600 [00:03<01:02,  9.07it/s]


31/31 [==============================] - 0s 2ms/step


ValueError: `axis` must be fewer than the number of dimensions (1)

In [ ]:
print(f"con_train_data shape: {con_train_data.shape}")
print(f"cat_train_data shape: {cat_train_data.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"con_val_data shape: {con_val_data.shape}")
print(f"cat_val_data shape: {cat_val_data.shape}")
print(f"y_val shape: {y_val.shape}")

con_train_data shape: (988, 7)
cat_train_data shape: (988, 59)
y_train shape: (988,)
con_val_data shape: (7,)
cat_val_data shape: (59,)
y_val shape: (247,)
